In [1]:
import time

from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


from src.gemini_llm import GeminiLLM
from src.utils import wiki_intro
from src.utils import train_test_split

Loading environment variables...
Enviroment vars loaded: True


In [2]:
model = GeminiLLM(model_name="gemini-2.0-flash-lite", temperature=0)

In [3]:
model.categorize(company_name="Altria Group")

'Food drink & tobacco'

In [5]:
model.categorize_get_description(
    company_name="Altria Group",
    get_description=wiki_intro
)

'Food drink & tobacco'

In [6]:
model.categorize(company_name="wewaealkedad")

[Warning] Response not in categories: Other


'Other'

Because I have limited API requests for the free plan with Gemini I only got results on the test set. This test set is also used for the other models.

In [6]:
df = pd.read_csv("../resources/dataset.csv")

In [7]:
train_df, test_df = train_test_split(df)

In [8]:
test_df

,COMPANY,CATEGORY
568,Pitney Bowes,Business services & supplies
1603,Nanto Bank,Banking
1010,Agfa-Gevaert,Household & personal products
1466,US Commercial,Diversified financials
968,Computer Associates,Software & services
...,...,...
1433,Tiffany,Retailing
697,Murata Manufacturing,Business services & supplies
139,Cendant,Hotels restaurants & leisure
1212,Canadian Tire,Retailing


A workaround for the rate limits of the Free tier Gemini API.

In [9]:
def safe_llm_call(company_name, model, get_description, max_retries=5):
    for attempt in range(max_retries):
        try:
            if get_description:
             return  model.categorize_get_description(
                        company_name=company_name,
                        get_description=wiki_intro
                )
            
            return model.categorize(company_name=company_name)
            
        except Exception as e:
            if "ResourceExhausted" in str(e) or "quota" in str(e).lower():
                wait_time = 60
                print(f"Rate limit hit. Retrying in {wait_time:.1f}s (attempt {attempt+1}/{max_retries})...")
                time.sleep(wait_time)
            else:
                print(f"Error with {company_name}: {e}")
                return "n/a"
    print(f"Max retries exceeded for {company_name}. Returning 'n/a'.")
    return "n/a"

In [10]:
safe_llm_call("Altria Group", model, get_description=True)

'Food drink & tobacco'

In [12]:
result_no_description = []
for name in tqdm(test_df["COMPANY"]):
    result_no_description.append(
        safe_llm_call(name, model, get_description=False)
    )

  1%|          | 4/500 [00:01<03:27,  2.39it/s]

[Warning] Response not in categories: Other


  6%|▌         | 31/500 [00:14<03:10,  2.46it/s]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 30
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 41
}
].


Rate limit hit. Retrying in 60.0s (attempt 1/5)...


 10%|█         | 52/500 [01:25<03:15,  2.29it/s]  

...

 96%|█████████▌| 480/500 [14:59<00:09,  2.19it/s]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 30
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 56
}
].


Rate limit hit. Retrying in 60.0s (attempt 1/5)...


100%|█████████▉| 498/500 [16:10<00:01,  1.90it/s]

[Warning] Response not in categories: Other


100%|██████████| 500/500 [16:10<00:00,  1.94s/it]


In [13]:
len(result_no_description)

500

In [15]:
test_df['PRED_CATEGORY_NO_DESC'] = result_no_description

In [17]:
np.mean(test_df['PRED_CATEGORY_NO_DESC'] == test_df['CATEGORY'])

0.776

In [ ]:
df.to_csv("../resources/dataset_with_predictions.csv", index=False)
# in case of crash

In [19]:
result_with_description = []
for name in tqdm(test_df["COMPANY"]):
    result_with_description.append(
        safe_llm_call(name, model, get_description=True)
    )

  0%|          | 2/500 [00:01<06:19,  1.31it/s]

[Fallback] Searching for: Agfa-Gevaert


  1%|          | 3/500 [00:02<08:27,  1.02s/it]

[Fallback] Searching for: US Commercial


  1%|          | 5/500 [00:05<08:19,  1.01s/it]

...

 98%|█████████▊| 491/500 [19:51<00:10,  1.12s/it]

[Warning] Response not in categories: Other


100%|█████████▉| 499/500 [19:56<00:00,  1.31it/s]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 30
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 2
}
].


Rate limit hit. Retrying in 60.0s (attempt 1/5)...


100%|██████████| 500/500 [21:00<00:00,  2.52s/it]


In [21]:
test_df['PRED_CATEGORY_WITH_DESC'] = result_with_description
np.mean(test_df['PRED_CATEGORY_WITH_DESC'] == test_df['CATEGORY'])

0.726

In [ ]:
# bootstrap standard error for confidence intervals
from src.eval import calculate_metrics, calculate_ste_metrics_with_bootstrap
from sklearn.preprocessing import LabelEncoder

    
y_true = test_df['CATEGORY'].values
# First, identify unique classes including the 'Other' class in predictions
all_classes = np.unique(np.concatenate([
    y_true,
    test_df['PRED_CATEGORY_NO_DESC'].values,
    test_df['PRED_CATEGORY_WITH_DESC'].values
]))

# Create a label encoder that includes all classes
le = LabelEncoder()
le.fit(all_classes)

# Encode all arrays
y_true_encoded = le.transform(y_true)
y_pred_no_desc = le.transform(test_df['PRED_CATEGORY_NO_DESC'].values)
y_pred_with_desc = le.transform(test_df['PRED_CATEGORY_WITH_DESC'].values)

# Now we can calculate metrics with properly encoded labels
metrics_no_desc = calculate_metrics(y_true_encoded, y_pred_no_desc)
metrics_with_desc = calculate_metrics(y_true_encoded, y_pred_with_desc)

# Calculate bootstrap standard errors
ste_no_desc = calculate_ste_metrics_with_bootstrap(y_true_encoded, y_pred_no_desc)
ste_with_desc = calculate_ste_metrics_with_bootstrap(y_true_encoded, y_pred_with_desc)

# Display results
metrics_no_desc, metrics_with_desc, ste_no_desc, ste_with_desc


({'f1': 0.7877742767754623,
  'precision': 0.7555683842009833,
  'recall': 0.7164840052454159,
  'accuracy': 0.776},
 {'f1': 0.744093026198812,
  'precision': 0.7506080085704225,
  'recall': 0.6540583380457584,
  'accuracy': 0.726},
 {'f1': 0.018394217175705593,
  'precision': 0.02224683289447747,
  'recall': 0.023030516303994213,
  'accuracy': 0.018341162885706034},
 {'f1': 0.020024022685492627,
  'precision': 0.028331083999983624,
  'recall': 0.025402954664524163,
  'accuracy': 0.019669155955454733})

In [51]:
# print nicely the results with bootstrap standard errors
for key in metrics_no_desc.keys():
    print(f"{key}: {metrics_no_desc[key]:.4f} ± {ste_no_desc[key]:.4f} (no desc)")
    print(f"{key}: {metrics_with_desc[key]:.4f} ± {ste_with_desc[key]:.4f} (with desc)")
    print()

# for key in metrics_with_desc.keys():
#     print(f"{key}: {metrics_with_desc[key]:.4f} ± {ste_with_desc[key]:.4f} (with desc)")
#     print(f"{key}: {metrics_no_desc[key]:.4f} ± {ste_no_desc[key]:.4f} (no desc)")
#     print()


f1: 0.7878 ± 0.0184 (no desc)
f1: 0.7441 ± 0.0200 (with desc)

precision: 0.7556 ± 0.0222 (no desc)
precision: 0.7506 ± 0.0283 (with desc)

recall: 0.7165 ± 0.0230 (no desc)
recall: 0.6541 ± 0.0254 (with desc)

accuracy: 0.7760 ± 0.0183 (no desc)
accuracy: 0.7260 ± 0.0197 (with desc)



In [25]:
np.sum(test_df['PRED_CATEGORY_WITH_DESC'] == 'n/a'), np.sum(test_df['PRED_CATEGORY_NO_DESC'] == 'n/a'),

(0, 0)

See how many of the prediction are the same.

In [26]:
np.mean(test_df['PRED_CATEGORY_NO_DESC'] == test_df['PRED_CATEGORY_WITH_DESC'])

0.808

In [28]:
mask = (test_df['PRED_CATEGORY_NO_DESC'] != test_df['PRED_CATEGORY_WITH_DESC']) & (test_df['PRED_CATEGORY_NO_DESC'] != 'n/a') \
    & (test_df['PRED_CATEGORY_WITH_DESC'] == test_df['CATEGORY'])
test_df[mask]

,COMPANY,CATEGORY,PRED_CATEGORY_NO_DESC,PRED_CATEGORY_WITH_DESC
754,MTR,Transportation,Hotels restaurants & leisure,Transportation
1034,Nippon Mining,Oil & gas operations,Materials,Oil & gas operations
1133,Apollo-Education Group,Business services & supplies,Other,Business services & supplies
1964,Valiant Holding,Banking,Insurance,Banking
1268,Avnet,Technology hardware & equipment,Trading companies,Technology hardware & equipment
642,MAN Group,Capital goods,Trading companies,Capital goods
1158,China Development Finl,Banking,Diversified financials,Banking
670,Capitalia,Banking,Other,Banking
1293,PMI Group,Insurance,Other,Insurance
1969,Performance Food,Food markets,Food drink & tobacco,Food markets


In [29]:
mask = (test_df['PRED_CATEGORY_NO_DESC'] != test_df['PRED_CATEGORY_WITH_DESC']) & (test_df['PRED_CATEGORY_NO_DESC'] != 'n/a') \
    & (test_df['PRED_CATEGORY_NO_DESC'] == test_df['CATEGORY'])

test_df[mask]

,COMPANY,CATEGORY,PRED_CATEGORY_NO_DESC,PRED_CATEGORY_WITH_DESC
568,Pitney Bowes,Business services & supplies,Business services & supplies,Technology hardware & equipment
218,Bouygues Group,Construction,Construction,Conglomerates
69,Comcast,Media,Media,Telecommunications services
1519,Kawasho,Trading companies,Trading companies,Media
1074,Iberia,Transportation,Transportation,Other
1462,SEM-Samsung Electro,Technology hardware & equipment,Technology hardware & equipment,Semiconductors
970,Daikin Industries,Capital goods,Capital goods,Technology hardware & equipment
935,Kobe Steel,Materials,Materials,Conglomerates
1314,Level 3 Commun,Telecommunications services,Telecommunications services,Other
348,EMC,Technology hardware & equipment,Technology hardware & equipment,Other


In [32]:
print(wiki_intro("Bouygues Group"))

[Fallback] Searching for: Bouygues Group
Bouygues S.A. (French pronunciation: [bwiɡ]) is a French engineering group headquartered in the 8th arrondissement of Paris, France. Bouygues is listed on the Euronext Paris exchange and is a blue chip in the CAC 40 stock market index. The company was founded in 1952 by Francis Bouygues and has been led by his son Martin Bouygues since 1989. Martin's older brother, Olivier Bouygues, is a board member.
The group specialises in construction (Colas Group and Bouygues Construction), real estate development (Bouygues Immobilier), media (TF1 Group), and telecommunications (Bouygues Telecom).


How much can we trust the labels? For some cases there is ambiguity. `Bouygues Group` description from Wikipedia says that it is a conglomerate.

In [31]:
print(wiki_intro("Comcast"))

Comcast Corporation, formerly known as Comcast Holdings, is an American multinational mass media corporation that works in telecommunications and entertainment. It is headquartered at the Comcast Center in Philadelphia. Comcast is the third-largest broadcasting and cable television company worldwide by revenue (behind China Mobile and Verizon). It is the third-largest pay-TV company, the second-largest cable TV company by subscribers, and the largest home Internet service provider in the United States. In 2023, the company was ranked 51st in the Forbes Global 2000.
Comcast is additionally the nation's third-largest home telephone service provider. It provides services to U.S. residential and commercial customers in 40 states and the District of Columbia. As the owner of NBCUniversal since 2013, Comcast is also a high-volume producer of films for theatrical exhibition and television programming, and a theme parks operator. It is the fourth-largest telecommunications company by worldwide

Same as the above case - here it explicitly says that it is a telecommunications company.

In [ ]:
def plot_confusion_matrix(y_true, y_pred, categories, normalize=True):
    cm = confusion_matrix(y_true, y_pred, labels=categories)
    
    # normalize -> percentages
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
        cm = np.nan_to_num(cm) * 100  # now in [0,100]
    
    fig, ax = plt.subplots(figsize=(15, 12))
    

    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=categories)
    disp.plot(cmap=plt.cm.Blues, ax=ax, values_format=".1f")

    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.grid(False)
    ax.set_title("Normalized Confusion Matrix (%)" if normalize else "Confusion Matrix", fontsize=16)
    plt.tight_layout()
    
categories = pd.read_csv("../resources/categories.csv")['CATEGORY'].unique().tolist()
plot_confusion_matrix(test_df['CATEGORY'], test_df['PRED_CATEGORY_NO_DESC'], categories, normalize=True)
plt.savefig("../resources/confusion_matrix_no_desc.svg", dpi=300, bbox_inches='tight')

In [39]:
cm = confusion_matrix(test_df['CATEGORY'], test_df['PRED_CATEGORY_NO_DESC'], labels=categories)
cm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
cm = np.nan_to_num(cm) * 100  # now in [0,100]
# find most confused categories
confused_categories = []
for i in range(len(categories)):
    for j in range(len(categories)):
        if i != j and cm[i, j] > 0:
            confused_categories.append((categories[i], categories[j], cm[i, j]))

In [40]:
confused_categories = sorted(confused_categories, key=lambda x: x[2], reverse=True)
confused_categories[:10]

[('Business services & supplies', 'Technology hardware & equipment', 40.0),
 ('Trading companies', 'Conglomerates', 40.0),
 ('Household & personal products', 'Consumer durables', 33.33333333333333),
 ('Food markets', 'Retailing', 28.57142857142857),
 ('Conglomerates', 'Capital goods', 25.0),
 ('Conglomerates', 'Aerospace & defense', 25.0),
 ('Technology hardware & equipment', 'Telecommunications services', 20.0),
 ('Trading companies', 'Retailing', 20.0),
 ('Construction', 'Materials', 17.647058823529413),
 ('Software & services', 'Business services & supplies', 16.666666666666664)]

In [ ]:
test_df.to_csv("../resources/dataset_with_predictions.csv", index=False)